# Sreality - Cleaning & Dropping
## Jirka Zelenka
### 12.3.-24.4.2020
### Celý projekt = Scraping + Cleaning & Dropping + Vizualizace + All_In_One + PowerBI
-----------------------------------------------------------------------------

### Prerekvizity:
* nainstalované package
* přístup k souboru "obce_okresy_kraje.xlsx" a "Adresy.xlsx"
* datový soubor z části SCRAPING - pracovní název "a4_SCRAPED_prodej_byty.xlsx"


### Obsah:
* 1) Importování Packagů
* 2) Úprava adres, Počet obyvatel
* 3) Update Adresáře
* 4) Cena za metr
* 5) Missing values
* 6) Extrémy
* 7) Duplikáty
* 8) Spuštění

-----------------------
# 1) Importování packagů
-----------------------

In [3]:

##### Obecné ############
import pandas as pd                     # for dataframes' manipulation
from pandas import DataFrame            # for creating dataframes
import numpy as np                      # for arrays
import matplotlib as plt                # for plotting
from matplotlib.pyplot import figure    # for saving and changing size of plots

from collections import Counter         # for counting elements 
from datetime import datetime           #for actual date
import re                               # !!! relativní Novinka - regular expressions
from time import sleep                  # for sleeping (slowing down) inside a function
import random                           # for random number (sleeping)
import math                             # Round float
import time                             # Time measuring
import itertools                        # for unlisting nested lists


##### Scraping ############
import requests                         # for robots check
from bs4 import BeautifulSoup           # for parsing
from selenium import webdriver          # for browsers control
import json                             # for Requests

##### GeoPy ############        
from geopy.geocoders import Nominatim   # Geolocator   # pip install geopy  
from geopy.exc import GeocoderTimedOut  # for Error handling

##########################
# Zaítm nepoužito:
##### Widgets ############
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import os
from IPython.display import Image

##### Bonus - Hudba ############
import winsound                        # for Beep-sounds

##### Vizualizace ############
import seaborn as sns                  #for cool plots



import sys                             # ???

-----------------------
# 2) Úprava adres, Počet obyvatel
-----------------------

In [19]:
# 1 = Mezery u Adres ###############################################################################################################################

############ Mezery před názvy (všeho) a pomlčky doplněné z GeoPy
def smaž_mezery(x):
    x = re.findall(r'\w+', str(x))   # Když napíšu tento regex (r'\w+-?\w+'), tak bych i zachoval "pomlčku", 
    x = ' '.join(x)                  # ale to já naopak nechci, vyřeším si dvě věci najednou 
    
    return x

In [20]:


# 2 = Posunutý Okres, Kraj... ################################################################################################################################
# Přidá kontrolní slpupec indikující, jestli je Kraj vyplněný slovem okres nebo ne

def kraj_check(x):
    x = x.split()[0]
    return x

# Podle kontrolního sloupce ověří  a případně posune hodnoty adres "doleva"
def uprav_adresy(x):

    x["kraj_check"] = x["kraj"].apply(kraj_check)

    # U těch řádků, kde byl okres v názvu kraje posuň všechny tři hodnoty "doleva"
    x.oblast[x["kraj_check"] == "okres"] = x.město[x["kraj_check"] == "okres"].copy()
    x.město[x["kraj_check"] == "okres"] = x.okres[x["kraj_check"] == "okres"].copy()
    x.okres[x["kraj_check"] == "okres"] = x.kraj[x["kraj_check"] == "okres"].copy()
    x.kraj[x["kraj_check"] == "okres"] = -1

    # Oříznutý dataset OBCE - externí
    obce = pd.read_excel(r"obce_okresy_kraje.xlsx")
    obce_kus = obce[["okres", "kraj"]].copy()

    # LEFT Merge by měl přiřadit okresům správné Kraje z df obcí
    data = pd.merge(x, obce_kus, on=['okres'], how = "left")

    # Smazání případných duplikátů
    data.drop_duplicates(keep = "first", inplace = True)

    # Přiřazení nového sloupečku na starý
    data.kraj_x[data["kraj_check"] == "okres"] = data.kraj_y[data["kraj_check"] == "okres"].copy()

    # Přejmenování a získání lepších sloupců
    data = data.rename(columns={'kraj_x': 'kraj'})
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "cena", "url", "url_id", "coords", "short_coords", "lat", "lon"]]  #Doplněno LAT a LON nově

    data.reset_index(inplace = True, drop = True)

    return data
# 3 = Počet obyvatel ###############################################################################################################################
# Z externího excelu, namapování počtu obyvatel podle města a okreuu a kraje (existují duplicitní města)

def počet_obyvatel(x):

    obce = pd.read_excel("obce_okresy_kraje.xlsx")
    obce_kus = obce[["město", "okres", "kraj", "obyvatelé"]].copy()
    data = pd.merge(x, obce_kus ,on = ["město", "okres", "kraj"], how = "left")
    data.drop_duplicates(keep = "first", inplace = True)

    return data


-----------------------
# 3) Update Adresáře
-----------------------

In [34]:
# 4 = Update databáze adres ###############################################################################################################################

def update_databáze_adres(x):
    
    # Načtení Adresáře
    adresy = pd.read_excel("Adresy.xlsx")
    len1 = len(adresy)
    print("-- Původní délka adresáře: " + str(len1))

    # Oříznu si data na to, co chci uložit a přidám aktuální datum
    data2 = x[["oblast", "město", "okres", "kraj", "url", "url_id", "short_coords"]]  # Odebráno Coords, vznikaly duplikáty zbytečně
    data2["Datum"] = datetime.now().strftime("%Y-%m-%d")

    # Připojím nová data, dropnu duplikáty (mažu odzadu), resetnu indexy, změřím délku
    # Přírůstek by měl odpovídat počtu řádků dohledaných přes Geopy minus duplikáty případně
    adresy = pd.concat([adresy, data2])
    adresy = adresy.drop_duplicates(subset = [ "url_id", "short_coords"], keep = "first", inplace = False)
    
    ### Nově vloženo: Odebrat shitty adresy - ty droppuju až dál, ale do Adresáře je tady poslat nechci
    adresy = adresy[adresy.short_coords != "(0.01, 0.01)"]
    
    # Změněna logika, dropni všechno kde máš dvojí varianty pro kombinaci url_id + short_coords, protože přesně tohohle se to mapuje a dělá to pak rozdvojení různá. !!
    # Jinak ponechává to možnost že mám jedno ID vícekrát s různými short_coords, ale to je možné, nebrání to zpětnému mapování
    adresy.reset_index(inplace = True, drop = True)
    len2 = len(adresy)
    print( "-- Nová délka adresáře: " + str(len2) + ", přidáno "+ str(len2-len1) + " záznamů. Ukládám do Adresy.xlsx")

    return adresy

-----------------------
# 4) Cena za metr
-----------------------

In [22]:
def cena_metr(y):

    y["cena_metr"] = y["cena"].astype(int)/y["plocha"].astype(int)
    y["cena_metr"] = y["cena_metr"].apply(lambda x: round(x, 1))
    
    return y

-----------------------
# 5) Missing values
-----------------------

In [23]:
def missing_values(x):
    
    data = x.copy()
    
    # Promazání:
    ## Nulových cen
    ## CHybějících ploch
    ## Chybjících adres (souřadnice neznámé)
    
    print("-- Celkem záznamů bez ceny: "+ str( len(data[data["cena"] == 0]) )   )
    print("-- Celkem záznamů bez popisu => plochy: "+ str( len(data[data["plocha"] == -1]) ) )
    print("-- Celkem záznamů bez okresu a kraje (pouze starší data): "+ str(len(data[data["kraj"] == "1"]))) 
    # Pokoje by měly být OK
    # Obyvatelé taky
    # Cena_metr pak už taky
    
    data = data[data.cena != 0]
    data = data[data.plocha != -1]
    data = data[data.kraj != "1"]
    
    
    return data

-----------------------
# 6) Extrémy
-----------------------

In [24]:
def extrémy(x):
    
    data = x.copy()     # Upraveno aby to korespondovalo s Tisíci, resp s původními cenami
    
    data = data[data.cena > 50000] # Zatím nikdy nebyl byt pod 100.000, resp za 90.000 půl byt
    data = data[data.plocha > 10] # Zatím nikdy nebyla plocha pod 12, snad je to rozumný předpoklad
    data = data[data.cena_metr < 450000]   # Zatím nikdy nebyla Víc než 360 za metr, jen v jednom chybném zadání rovnou 568 !!
    data = data[data.cena_metr > 1000]   #Toto se občas vidí a je to očividná chyba v zadání inzerátu. Viz CHlumec - 376m !! 338.000

    return data


-----------------------
# 7) Duplikáty
-----------------------

In [25]:

def duplikáty(x):
    
    #Po několika pokusech a hledání různých variant duplikátů,
    # volím variantu smazat inzeráty, které jsou ve všem identické (krom url a url_id - ty jsou samé unikátní).
    # Pár by se našlo podobných, asi zdvojných, co mají třeba jen odchylku v souřadnici nebo v ceně. ale jej jich málo, a byl oby to as ivíc práce než užitku.
    
    data =  x.copy()
    a = len(data[data.duplicated([ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "coords", "short_coords", "lat", "lon"],keep= "first")])
    print("-- Počet řádků, které k sobě mají duplikáty (krom url a url_id): " + str(a))                                                                                                         
    data = data.drop_duplicates(subset =[ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "coords", "short_coords", "lat", "lon" ],keep= "first", inplace = False) 
    #Z dat dropnu duplikáty podle všeho krom url a url_id, nechávám si první záznamy
    
    return data

-----------------------
# 8) Spuštění
-----------------------

In [26]:

################################################################################################################################
##############################################   CLEANING    ###################################################################
################################################################################################################################

def clean_all(x):
    
    data = pd.read_excel(x)   # Pracovní verze s načítáním dat, jinak napojeno na Scrap
    #data = x.copy()
    
    print("----------------")
    print("Počet záznamů před čištěním: " + str(len(data)))
    
    
    # 1 = Mezery u adres 
    data["oblast"] = data["oblast"].apply(smaž_mezery)
    data["město"] = data["město"].apply(smaž_mezery)
    data["okres"] = data["okres"].apply(smaž_mezery)
    data["kraj"] = data["kraj"].apply(smaž_mezery)
    print("1/6 Vymazány mezery před názvy, následuje posouvání rozbitých okresů a krajů")
    
    # 2 = Posunutý Okres, Kraj...
    data = uprav_adresy(data)
    print("2/6 Posunuty rozbité okresy a kraje, následuje doplnění počtu obyvatel.")
     
    # 3 = Počet obyvatel z Excelu
    data = počet_obyvatel(data)
    data.to_excel("a5_Adresy_a_obyvatele_data_prodej_byty.xlsx")
    print("-- Aktuální délka datasetu: " + str(len(data)))
    print("3/6 Doplněn počet obyvatel, updatuji databázi adres.")
    
    # 4 = Update databáze adres 
    adresy = update_databáze_adres(data)
    adresy.to_excel(r"Adresy.xlsx")
    print("4/6 Databáze Adres updatována. Délka datasetu: " + str(len(data)) + " záznamů. Převádím ceny na tisíce")    

    # 5 Cena na tisíce - zatím vypínám kvůli POWER BI
    #data = cena_tisíce(data)
    print("5/6 Ceny NECHCI ABY BYLY převedeny na tisíce. Počítám ceny za metr.")
    
    # 6 Cena za metr
    data = cena_metr(data)
    data["datum"] = datetime.datetime.now().strftime("%d.%m.%Y")   # Nově přidáno kvůli souhrnným datům
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "url", "url_id", "coords", "short_coords", "lat", "lon", "datum"]]
    data.to_excel("a6_CLEANED_data_prodej_byty.xlsx")
    print("6/6 Spočítány ceny za metr. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 2.")
    
          
    return data


In [27]:

################################################################################################################################
##############################################   DROPPING    ###################################################################
################################################################################################################################

def drop_all(x):
    
    #data = pd.read_excel(x)   # Pracovní verze s načítáním dat, jinak napojeno na Cleaning
    data = x.copy()
    x = len(data)
    print("----------------")
    print("Počet záznamů před čištěním: " + str(x))
    
    
    # 1 = Missing Values
    data = missing_values(data)
    y = len(data)
    print("1/3 Vymazány chybějící hodnoty (Ceny, Plochy, (Kraje)), celkem " + str(x-y) + " řádků. Zbývá " + str(y) + " záznamů.")

    # 2 = Extrémy
    data = extrémy(data)
    z = len(data)
    print("2/3 Vymazány extrémní hodnoty (Ceny, Plochy, Cena za metr, celkem " + str(y-z) + " řádků. Zbývá " + str(z) + " záznamů.")
    
    # 3 = Duplikáty
    data = duplikáty(data)
    b = len(data)
    data.to_excel("a7_DROPPED_data_prodej_byty.xlsx")
    print("3/3 Vymazány duplikáty, zbývá " + str(b) + " záznamů. Konec Fáze 3.")
    
    display(data.head())
    
    return data

In [ ]:
cleaned = clean_all("a4_SCRAPED_prodej_byty.xlsx")

In [ ]:
dropped = drop_all(cleaned)